In [1]:
import scanpy as scp
import numpy as np
import scipy.sparse
import pandas as pd
from datetime import datetime

In [2]:
%load_ext rpy2.ipython

Error importing in API mode: ImportError("dlopen(/Users/gatocor/miniforge3/envs/scrnaseq_vitamin_a/lib/python3.13/site-packages/_rinterface_cffi_api.abi3.so, 0x0002): Library not loaded: /Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib\n  Referenced from: <688DB777-6784-3A5B-A816-CC26D6DADD8D> /Users/gatocor/miniforge3/envs/scrnaseq_vitamin_a/lib/python3.13/site-packages/_rinterface_cffi_api.abi3.so\n  Reason: tried: '/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file), '/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file)")
Trying to import in ABI mode.
/Users/gatocor/miniforge3/envs/scrnaseq_vitamin_a/lib/python3.13/site-packages/rpy2/rinterface/__init__.py:1185: UserWarning: Environment variable "PWD" redefined by R and overriding exis

In [3]:
%R data <- readRDS("data_other_studies/prisca/GSE229513_gastruloidsobject.rds")

R callback write-console: Cargando paquete requerido: SeuratObject
  
R callback write-console: Cargando paquete requerido: sp
  
R callback write-console: 
Adjuntando el paquete: ‘SeuratObject’

  
R callback write-console: Els següents objectes estan emmascarats des de ‘package:base’:

    intersect, t

  


<rpy2.robjects.methods.RS4 object at 0x14d647c10> [25]
R classes: ('Seurat',)

In [7]:
%R umap_coords <- Embeddings(data, reduction = "umap")
%R -o umap_coords

In [8]:
%R obs <- data@meta.data
%R -o obs

In [5]:
%R raw_counts <- data[["RNA"]]@counts  # Sparse matrix of genes x cells
%R -o raw_counts

# Extract slots
i = np.array(raw_counts.do_slot("i"))       # row indices
p = np.array(raw_counts.do_slot("p"))       # column pointers
x = np.array(raw_counts.do_slot("x"))       # non-zero values
dims = tuple(raw_counts.do_slot("Dim"))     # (n_rows, n_cols)
raw_counts_sparse = scipy.sparse.csc_matrix((x, i, p), shape=dims)

In [6]:
%R gene_names <- rownames(raw_counts)
%R -o gene_names

In [11]:
adata = scp.AnnData(X=raw_counts_sparse.transpose(), obs=obs, var=pd.DataFrame(index=gene_names))
adata.obsm["X_umap"] = umap_coords
adata.var_names = gene_names
adata.obs.columns = adata.obs.columns.values.astype(str)
adata.layers["counts"] = adata.X.copy()
adata.layers["logcounts"] = np.log1p(adata.X)

In [12]:
adata.write_h5ad("data_other_studies/prisca/adata.h5ad")

In [13]:
adata

AnnData object with n_obs × n_vars = 71005 × 27027
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'batch', 'nCount_SCT', 'nFeature_SCT', 'integrated_snn_res.0.8', 'seurat_clusters', 'timepoints', 'integrated_snn_res.1.7', 'integrated_snn_res.0.5', 'integrated_snn_res.1.1', 'celltypeannotation'
    obsm: 'X_umap'
    layers: 'counts', 'logcounts'

## Save analysis script

In [14]:
# Get the current date
current_date = datetime.now().strftime("%Y%m%d")

# Construct the output file name with the current date
input_file = f"setup_priscas_data.ipynb"
output_file = f"script_versions/setup_priscas_data_{current_date}"

# Convert the notebook to HTML with the new file name
!jupyter nbconvert --to html "$input_file" --output "$output_file"

[NbConvertApp] Converting notebook setup_priscas_data.ipynb to html
[NbConvertApp] Writing 286994 bytes to script_versions/setup_priscas_data_20250602.html
